<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/colab_kernel_launcher_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qiskit scikit-learn numpy pandas matplotlib

In [ ]:
#!/usr/bin/env python3
"""
colab_kernel_launcher.py

Full pipeline for quantum-kernel class-imbalance experiments,
compatible with Colab/Jupyter (ignores unknown flags).

Features:
  - Generates 2D synthetic data at varying class-imbalance ratios
  - Builds ZZ and Pauli quantum feature maps at depths [1, 3, 5]
  - Computes fidelity² kernels and centers them
  - Measures full and mini-batch kernel–target alignment
  - Trains SVMs (unweighted vs. class-weighted) on precomputed kernels
  - Prints metrics, saves results to CSV, exports fidelity² histograms as PNG
"""

import os
import warnings
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from qiskit.circuit.library import zz_feature_map, PauliFeatureMap
from qiskit.quantum_info import Statevector, state_fidelity

warnings.filterwarnings("ignore", category=DeprecationWarning)


def generate_data(n_samples, imbalance, random_state=None):
    weights = [0.5, 0.5] if imbalance >= 1.0 else [imbalance, 1.0 - imbalance]
    return make_classification(
        n_samples=n_samples,
        n_features=2,
        n_redundant=0,
        n_clusters_per_class=1,
        weights=weights,
        class_sep=2.0,
        random_state=random_state
    )


def build_feature_map(name, reps):
    if name == "ZZ":
        return zz_feature_map(
            feature_dimension=2, reps=reps,
            entanglement='circular', insert_barriers=False
        )
    if name == "Pauli":
        return PauliFeatureMap(
            feature_dimension=2, reps=reps,
            paulis=['X', 'Y', 'Z'],
            entanglement='circular', insert_barriers=False
        )
    raise ValueError(f"Unknown feature map: {name}")


def compute_fidelity_kernel(fmap, X):
    states = [Statevector.from_instruction(fmap.assign_parameters(x)) for x in X]
    n = len(states)
    K = np.zeros((n, n), float)
    for i in range(n):
        for j in range(i, n):
            f2 = state_fidelity(states[i], states[j]) ** 2
            K[i, j] = K[j, i] = f2
    return K


def center_kernel(K):
    n = K.shape[0]
    one_n = np.ones((n, n)) / n
    return K - one_n @ K - K @ one_n + one_n @ K @ one_n


def kernel_target_alignment(Kc, y):
    yy = np.outer(y, y)
    num = np.trace(Kc @ yy)
    den = np.linalg.norm(Kc, 'fro') * np.linalg.norm(yy, 'fro')
    return num / den if den > 0 else 0.0


def mini_batch_alignment(K, y, batch_size=25, n_batches=100, random_state=None):
    rng = np.random.default_rng(random_state)
    y_signed = 2 * y - 1
    aligns = []
    for _ in range(n_batches):
        idx = rng.choice(len(y), size=min(batch_size, len(y)), replace=False)
        Kb = K[np.ix_(idx, idx)]
        yyb = np.outer(y_signed[idx], y_signed[idx])
        num = np.trace(Kb @ yyb)
        den = np.linalg.norm(Kb, 'fro') * np.linalg.norm(yyb, 'fro')
        aligns.append(num / den if den > 0 else 0.0)
    return np.mean(aligns)


def svm_cv(K, y, weighted=False, max_splits=5):
    classes, counts = np.unique(y, return_counts=True)
    n_splits = min(max_splits, counts.min())
    if len(classes) < 2 or n_splits < 2:
        return np.array([])
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    svc = SVC(kernel='precomputed', class_weight='balanced' if weighted else None)
    return cross_val_score(svc, K, y, cv=cv)


def plot_and_save_histogram(K, y, title, out_path, bins=30):
    idx = np.triu_indices_from(K, k=1)
    vals = K[idx]
    yy = np.outer(y, y)
    same = vals[yy[idx] == 1]
    cross = vals[yy[idx] != 1]

    plt.figure(figsize=(6, 4))
    plt.hist(same, bins=bins, alpha=0.6, label='same-class')
    plt.hist(cross, bins=bins, alpha=0.6, label='cross-class')
    plt.xlabel('Fidelity²')
    plt.ylabel('Count')
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()


def main():
    parser = argparse.ArgumentParser(
        description="Quantum Kernel Imbalance Analysis (Colab-compatible)"
    )
    parser.add_argument(
        "--n_samples", type=int, default=50,
        help="Samples per imbalance level"
    )
    parser.add_argument(
        "--outdir", type=str, default="results",
        help="Directory for CSV & histograms"
    )

    # ignore unknown flags in Jupyter/Colab
    args, _ = parser.parse_known_args()

    os.makedirs(args.outdir, exist_ok=True)
    np.random.seed(42)

    imbalance_levels = [1.0, 0.75, 0.5, 0.2, 0.1]
    feature_maps    = ["ZZ", "Pauli"]
    depths          = [1, 3, 5]
    results         = []

    for imb in imbalance_levels:
        X, y = generate_data(args.n_samples, imb, random_state=1)
        print(f"\n=== Imbalance {int(imb*100)}% (N={args.n_samples}) ===")

        for name in feature_maps:
            for d in depths:
                fmap = build_feature_map(name, d)
                K    = compute_fidelity_kernel(fmap, X)
                Kc   = center_kernel(K)

                full_al = kernel_target_alignment(Kc, y)
                mb_al   = mini_batch_alignment(
                    K, y, batch_size=25, n_batches=100, random_state=0
                )
                scores   = svm_cv(K, y, weighted=False)
                scores_b = svm_cv(K, y, weighted=True)
                fmt      = lambda s: f"{s.mean():.3f}±{s.std():.3f}" if s.size else "n/a"

                print(
                    f"{name:6s} depth={d} | full_align={full_al:.3f}"
                    f" | mb_align={mb_al:.3f}"
                    f" | SVM={fmt(scores)}"
                    f" | SVM_bal={fmt(scores_b)}"
                )

                hist_filename = f"hist_{int(imb*100)}_{name}_d{d}.png"
                plot_and_save_histogram(
                    K, y,
                    title=f"{name} d={d}, Imbalance={int(imb*100)}%",
                    out_path=os.path.join(args.outdir, hist_filename)
                )

                results.append({
                    "imbalance":     imb,
                    "feature_map":   name,
                    "depth":         d,
                    "full_align":    full_al,
                    "mb_align":      mb_al,
                    "svm_mean":      scores.mean()   if scores.size else np.nan,
                    "svm_std":       scores.std()    if scores.size else np.nan,
                    "svm_bal_mean":  scores_b.mean() if scores_b.size else np.nan,
                    "svm_bal_std":   scores_b.std()  if scores_b.size else np.nan
                })

    # write CSV
    df = pd.DataFrame(results)
    csv_path = os.path.join(args.outdir, "results.csv")
    df.to_csv(csv_path, index=False)
    print(f"\nResults saved to {csv_path}")


if __name__ == "__main__":
    main()